In [1]:
import numpy as np
import pandas as pd
import sklearn
import os
import math
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

In [4]:
data = pd.read_csv('/content/data_lists.csv')
data['tau'] = data['T'] - (data['t']*data['T']/10)
data['moneyness'] = [data['asset'][i]/data['k'][i] for i in range(data.shape[0])]

In [5]:
data.shape

(225925, 14)

In [ ]:
def d1(S,K,r,sigma,tau):
    return (np.log(S/K) + (r+(0.5*(sigma**2)))*tau)/(sigma*np.sqrt(tau))

def d2(S,K,r,sigma,tau):
    d_1= d1(S,K,r,sigma,tau)
    return d_1-(sigma*np.sqrt(tau))

def delta(S,K,r,sigma,tau,d1,d2):
    return norm.cdf(d1)

def gamma(S,K,r,sigma,tau,d1,d2):
    return (1/np.sqrt(2*math.pi))*np.exp(-(d1**2)/2)*(1/(sigma*S*np.sqrt(tau)))

def theta(S,K,r,sigma,tau,d1,d2):
    return -(((S*sigma)/(2*np.sqrt(tau)))*(1/np.sqrt(2*math.pi))*np.exp(-(d1**2)/2))-(r*K*np.exp(-r*(tau)))*norm.cdf(d2)

def vega(S,K,r,sigma,tau,d1,d2):
    return S*np.sqrt(tau)*(1/np.sqrt(2*math.pi))*np.exp(-(d1**2)/2)

def rho(S,K,r,sigma,tau,d1,d2):
    return (tau*K*np.exp(-r*tau)*norm.cdf(d2))

In [ ]:
data['d1'] = list(map(d1,data['asset'],data['k'],data['r'],data['sigma'],data['tau']))
data['d2'] = list(map(d2,data['asset'],data['k'],data['r'],data['sigma'],data['tau']))
data['delta'] = list(map(delta,data['asset'],data['k'],data['r'],data['sigma'],data['tau'],data['d1'],data['d2']))
data['gamma'] = list(map(gamma,data['asset'],data['k'],data['r'],data['sigma'],data['tau'],data['d1'],data['d2']))
data['theta'] = list(map(theta,data['asset'],data['k'],data['r'],data['sigma'],data['tau'],data['d1'],data['d2']))
data['vega'] = list(map(vega,data['asset'],data['k'],data['r'],data['sigma'],data['tau'],data['d1'],data['d2']))
data['rho'] = list(map(rho,data['asset'],data['k'],data['r'],data['sigma'],data['tau'],data['d1'],data['d2']))

In [ ]:
data.drop(inplace=True, axis=1, columns = ['Unnamed: 0'])
data.head()

,r,sigma,T,s0,k,t,asset,call,asset_greater_call,scaled_call,scaled_asset,tau,moneyness,d1,d2,delta,gamma,theta,vega,rho
0,0.05,0.05,0.1,10.0,10.0,0.0,10.000000,0.090978,True,0.009098,1.0,0.10,1.000000,0.324133,0.308322,0.627082,2.394010,-0.608243,1.197005,0.617984
1,0.05,0.05,0.1,10.0,10.0,1.0,9.977024,0.071240,True,0.007124,1.0,0.09,0.997702,0.154153,0.139153,0.561255,2.634254,-0.604191,1.179973,0.497558
2,0.05,0.05,0.1,10.0,10.0,2.0,9.979236,0.066344,True,0.006634,1.0,0.08,0.997924,0.142939,0.128797,0.556831,2.798086,-0.622830,1.114591,0.439232
3,0.05,0.05,0.1,10.0,10.0,3.0,10.085137,0.133263,True,0.013326,1.0,0.07,1.008514,0.912040,0.898811,0.819126,1.972797,-0.657203,0.702286,0.568942
4,0.05,0.05,0.1,10.0,10.0,4.0,9.964273,0.045868,True,0.004587,1.0,0.06,0.996427,-0.041160,-0.053407,0.483584,3.266261,-0.644005,0.972889,0.286362


In [ ]:
data.to_csv('data/final_data.csv', index=False)